In [222]:
import os
import json
import pandas as pd
from gensim.models.fasttext import FastText

In [223]:
paths = ["data/kho/" + x for x in os.listdir("data/kho")]
paths += ["data/kko/" + x for x in os.listdir("data/kko")]
paths += ["data/sd/" + x for x in os.listdir("data/sd")]

In [224]:
data = []

for i in range(0, len(paths)):
    with open(paths[i], 'r') as f:
        data += json.load(f)
        
df = pd.DataFrame(data)
del data

In [225]:
df.sort_values("year", inplace = True)
df.reset_index(inplace = True)
df.drop("index", 1, inplace = True)

In [226]:
df['text'] = df['text'].apply(lambda x: x.split())

In [227]:
texts = df['text']
texts

0        [oikeudenkäymiskaari, oikeudenkäymiskaari, luk...
1        [kauppakaari, kauppakaari, lukukaupasta, ja, v...
2        [rakennuskaari, rakennuskaari, luku, luku, on,...
3        [maakaari, maakaari, luku, maan, huoneen, ja, ...
4        [konkurssisääntö, kumottu, konkurssisääntö, ku...
                               ...                        
62405    [laki, kalastuksesta, tenojoen, vesistössä, no...
62406    [laki, ilmailulain, muuttamisesta, laki, ilmai...
62407    [laki, henkilötietojen, käsittelystä, tullissa...
62408    [laki, koulumatkatukilain, muuttamisesta, laki...
62409    [rakennusurakan, tilaaja, oli, lähettänyt, ura...
Name: text, Length: 62410, dtype: object

In [228]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py
model = FastText(sentences = texts, window = 5, size = 100, min_count = 5)

In [229]:
print(model)

FastText(vocab=208397, size=100, alpha=0.025)


In [260]:
word = "koronavirus"
print("\'" + word + "\'" + " included in vocabulary:")
print(word in model.wv.vocab)
print("")
print("Most similar words:")
model.wv.most_similar(word)

'koronavirus' included in vocabulary:
False

Most similar words:


[('potassium', 0.7814205884933472),
 ('fosforipentakloridi', 0.7703386545181274),
 ('tritium', 0.7698299884796143),
 ('emepronium', 0.7675131559371948),
 ('phleum', 0.7662776708602905),
 ('delphinium', 0.7641345262527466),
 ('triticum', 0.7605825662612915),
 ('strontium', 0.7605584859848022),
 ('esculentum', 0.7595813274383545),
 ('argentum', 0.75655597448349)]

In [241]:
model.save("models/full_model")
# model = FastText.load("models/test_model")